In [2]:
import os
import traceback
import sys


In [3]:
from dotenv import load_dotenv

In [22]:
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_pinecone import PineconeVectorStore
from langchain.chains import RetrievalQA



In [4]:
from pinecone import Pinecone

In [5]:
from PyPDF2 import PdfReader

In [6]:
file_path=r"E:\ChatwithFile\Einstein_Relativity.pdf"

In [7]:
reader= PdfReader(file_path)
text = ""
for page in reader.pages:
    text += page.extract_text() + "\n"
    

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_text(text)

In [9]:
len(texts)

265

In [43]:
import os
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings

# Make sure this key is correct
os.environ["PINECONE_API_KEY"] = "pcsk_uaU9S_CWapvK8KHah1pScTmD8eXYL1nbyAFUizj6PAeCMrtzrpL6hJL1PAVsJjNsKowT5"

# Initialize Pinecone client
pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])


index = "chatwithfile"  # ✅ use the correct index name
 # ✅ passes the index name directly

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Connect to Pinecone index
vector_store = PineconeVectorStore(index_name=index, embedding=embeddings)


UnauthorizedException: (401)
Reason: Unauthorized
HTTP response headers: HTTPHeaderDict({'x-pinecone-api-version': '2025-04', 'x-cloud-trace-context': 'cd9dd9d82208338e0852c9df4f6d7fca', 'date': 'Tue, 19 Aug 2025 04:14:13 GMT', 'content-type': 'text/html', 'server': 'Google Frontend', 'Content-Length': '15', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: Invalid API Key


In [28]:
load_dotenv()

True

In [29]:
pinecone_api_key =os.getenv("PINECONE_API_KEY")
pinecone_index =os.getenv("PINECONE_INDEX")


In [33]:
print(os.getenv("PINECONE_API_KEY"))
print(pinecone_api_key)
print(pinecone_index)

pcsk_uaU9S_CWapvK8KHah1pScTmD8eXYL1nbyAFUizj6PAeCMrtzrpL6hJL1PAVsJjNsKowT5
pcsk_uaU9S_CWapvK8KHah1pScTmD8eXYL1nbyAFUizj6PAeCMrtzrpL6hJL1PAVsJjNsKowT5
chatwithfile


In [31]:
pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])

# Pick your index
index_name = "chatwithfile"

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vector_store = PineconeVectorStore(
    index=pc.Index(index_name),
    embedding=embeddings
)

UnauthorizedException: (401)
Reason: Unauthorized
HTTP response headers: HTTPHeaderDict({'x-pinecone-api-version': '2025-04', 'x-cloud-trace-context': 'b914b0fe988d21344d7cca105d073614', 'date': 'Tue, 19 Aug 2025 04:05:22 GMT', 'content-type': 'text/html', 'server': 'Google Frontend', 'Content-Length': '15', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: Invalid API Key


In [26]:
prompt = ChatPromptTemplate.from_template(
    """Answer the question based only on the following context: {context}

    Question: {question}
    Answer:"""
)

In [27]:
retriever = vector_store.as_retriever()

In [28]:
from langchain_core.runnables import RunnableMap, RunnablePassthrough

In [ ]:
rag_chain = (
    RunnableMap({
        "context": retriever,
        "question": RunnablePassthrough()
    })
    | prompt
    | llm
)

In [32]:
result = rag_chain.invoke("What is the special theory of relativity?")

In [33]:
print(result.content)

The special theory of relativity, according to Document(id='a8908772-59f7-427d-89eb-65cc87ba1fa5', has reference to Galileian domains, i.e. to those in which no gravitational field exists. In this connection a Galileian reference-body serves as body of reference, i.e. a rigid body the state of motion of which is so chosen that the Galileian law of the uniform rectilinear motion of “isolated” material points holds relatively to it.
